In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
datasets_path = "../datasets_for_ml/"

reservas_futuras = pd.read_csv(datasets_path+'reservas_futuras.csv')
transacoes = pd.read_csv(datasets_path+'transacoes.csv')
cancelados = pd.read_csv(datasets_path+'cancelados.csv')
faltantes = pd.read_csv(datasets_path+'faltantes.csv')

Vimos na análise dos dados que há interseções entre as tabelas de transações, cancelados e faltantes. Entretanto, suspeitamos que essas tabelas deveriam ser mutuamente exclusivas, de tal forma que não deveria haver interseções entre elas.

Vamos tomar a seguinte estratégia para lidar com esse problema:

* Todas as interseções de cancelados e faltantes com transações serão removidos, deixando as instâncias somente na tabela transações. Isso pois se o cliente está na tabela de transações significa que ele recebeu o serviço, portanto, não cancelou e nem faltou.

* Os clientes que estão na interseção de cancelados e faltantes serão removidos da tabela de faltantes e mantidos na tabela de cancelados. Se o cliente está na lista de cancelados, significa que ele cancelou e, portanto, não faltou.

Criando flags para auxiliar na remoção das instâncias

In [14]:
# Apenas para controle
transacoes.shape , cancelados.shape , faltantes.shape

((1862, 13), (240, 7), (59, 4))

In [15]:
cancelados['to_remove'] = cancelados.index
faltantes['to_remove'] = faltantes.index

In [16]:
remove = pd.merge(left = cancelados[['Booking Date','Code','to_remove']], 
               right = transacoes[['Date','Client']],
               left_on = ['Code','Booking Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre cancelados e transacoes: {len(remove)}")

cancelados = cancelados.drop(remove)



Quantidade de interseções entre cancelados e transacoes: 56


In [17]:
remove = pd.merge(left = faltantes[['Date','Code','to_remove']], 
               right = transacoes[['Date','Client']],
               left_on = ['Code','Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre faltantes e transacoes: {len(remove)}")
faltantes = faltantes.drop(remove)

Quantidade de interseções entre faltantes e transacoes: 1


In [18]:
remove = pd.merge(left = cancelados[['Booking Date','Code']], 
               right = faltantes[['Date','Code','to_remove']],
               left_on = ['Code','Booking Date'], 
               right_on=['Code','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre cancelados e transacoes: {len(remove)}")
faltantes = faltantes.drop(remove)

Quantidade de interseções entre cancelados e transacoes: 4


In [19]:
cancelados = cancelados.drop('to_remove',axis=1)
faltantes = faltantes.drop('to_remove',axis=1)

In [20]:
# Apenas para controle
transacoes.shape , cancelados.shape , faltantes.shape

((1862, 13), (184, 7), (54, 4))

Agora vemos que não há mais interseções entre as tabelas

In [21]:
pd.merge(left = cancelados, 
               right = transacoes,
               left_on = ['Code','Booking Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True)

,Cancel Date,Code,Service,Staff_x,Booking Date,Canceled By,Days,Receipt,Date,Description,...,Staff_y,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n,_merge


In [22]:
pd.merge(left = faltantes, 
               right = transacoes,
               left_on = ['Code','Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True)

,Date,Code,Service,Staff_x,Receipt,Description,Client,Staff_y,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n,_merge


In [23]:
pd.merge(left = cancelados, 
               right = faltantes,
               left_on = ['Code','Booking Date'], 
               right_on=['Code','Date'],
               how='inner',
               indicator=True)

,Cancel Date,Code,Service_x,Staff_x,Booking Date,Canceled By,Days,Date,Service_y,Staff_y,_merge


## **Construção do dataset para treinamento, teste e validação**